In [ ]:
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy as sci
from scipy import stats
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
#https://www.sc-best-practices.org/preprocessing_visualization/quality_control.html

def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

In [ ]:
sc.logging.print_versions()
sc.set_figure_params(facecolor="white", figsize=(8, 8))
sc.settings.verbosity = 3
sc._settings.settings._vector_friendly=True

## Reading the data



#### Visium filtered files

In [ ]:
adata_control_1 = sc.read_visium(path="/PATH/Control_1/",
                       count_file="filtered_feature_bc_matrix.h5")
adata_control_2 = sc.read_visium(path="/PATH/Control_2/",
                       count_file="filtered_feature_bc_matrix.h5")

adata_Treatment_1 = sc.read_visium(path="/PATH/Treatment_1/",
                       count_file="filtered_feature_bc_matrix.h5")
adata_Treatment_2 = sc.read_visium(path="/PATH/Treatment_2/",
                       count_file="filtered_feature_bc_matrix.h5")


#Makes the index unique by appending a number string to each duplicate index element: ‘1’, ‘2’, etc.
adata_control_1.var_names_make_unique()
adata_control_2.var_names_make_unique()

adata_Treatment_1.var_names_make_unique()
adata_Treatment_2.var_names_make_unique()

#### Read manual annotation

In [ ]:
annot_control_1 = pd.read_csv("Tumors_S1.csv")
annot_control_2 = pd.read_csv("Tumors_S2.csv")
annot_Treatment_1 = pd.read_csv("Tumors_S3.csv")
annot_Treatment_2 = pd.read_csv("Tumors_S4.csv")
#annot_control_1.rename(index={"Barcode":"Barcode","Tumor": "Tumors"})
#annot_control_2.rename(column={"Barcode":"Barcode","All tumors": "Tumors"})

In [ ]:
annot_control_1_nodules = annot_control_1.Barcode[annot_control_1.Tumor=="Nodules"].tolist()
annot_control_2_nodules = annot_control_2.Barcode[annot_control_2['All tumors']=="Nodules"].tolist()
annot_Treatment_1_nodules = annot_Treatment_1.Barcode[annot_Treatment_1.Tumors=="Nodules"].tolist()
annot_Treatment_2_nodules = annot_Treatment_2.Barcode[annot_Treatment_2.Tumors=="Nodules"].tolist()


In [ ]:
#Add barcode column
adata_control_1.obs["barcode"] = adata_control_1.obs_names
adata_control_2.obs["barcode"] = adata_control_2.obs_names
adata_Treatment_1.obs["barcode"] = adata_Treatment_1.obs_names
adata_Treatment_2.obs["barcode"] = adata_Treatment_2.obs_names

In [ ]:
#Filter only nodules
adata_control_1 = adata_control_1[adata_control_1.obs['barcode'].isin(annot_control_1_nodules)]
adata_control_2 = adata_control_2[adata_control_2.obs['barcode'].isin(annot_control_2_nodules)]

adata_Treatment_1 = adata_Treatment_1[adata_Treatment_1.obs['barcode'].isin(annot_Treatment_1_nodules)]
adata_Treatment_2 = adata_Treatment_2[adata_Treatment_2.obs['barcode'].isin(annot_Treatment_2_nodules)]

## QC and preprocessing

#### Calculate QC metrics

In [ ]:
# mitochondrial genes
adata_control_1.var["mt"] = adata_control_1.var_names.str.startswith("mt-")
adata_control_2.var["mt"] = adata_control_2.var_names.str.startswith("mt-")

adata_Treatment_1.var["mt"] = adata_Treatment_1.var_names.str.startswith("mt-")
adata_Treatment_2.var["mt"] = adata_Treatment_2.var_names.str.startswith("mt-")
# ribosomal genes
#adata.var["ribo"] = adata.var_names.str.startswith(("Rps", "Rpl"))
# hemoglobin genes.
#adata.var["hb"] = adata.var_names.str.contains(("^Hb[^(p)]"))
#sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)
sc.pp.calculate_qc_metrics(adata_control_1, qc_vars=["mt"], inplace=True,  log1p=True)
sc.pp.calculate_qc_metrics(adata_control_2, qc_vars=["mt"], inplace=True,  log1p=True)

sc.pp.calculate_qc_metrics(adata_Treatment_1, qc_vars=["mt"], inplace=True,  log1p=True)
sc.pp.calculate_qc_metrics(adata_Treatment_2, qc_vars=["mt"], inplace=True,  log1p=True)


In [ ]:
import matplotlib
pdf = matplotlib.backends.backend_pdf.PdfPages("output_nodulesonly_preQC.pdf")

for name, adata in [
    ("control_1", adata_control_1),
    ("control_2", adata_control_2),
    ("Treatment_1", adata_Treatment_1),
    ("Treatment_2", adata_Treatment_2),
]:
    fig, axs = plt.subplots(1, 5, figsize=(12, 3))
    fig.suptitle(f"Covariates for filtering: {name}")

    sns.histplot(adata.obs["total_counts"], kde=False, ax=axs[0],color='#BBBCFF').grid(False)
    axs[0].set_xlim(0,85000)
    sns.histplot(adata.obs["total_counts"][adata.obs["total_counts"] < 20000],kde=False,bins=40,ax=axs[1],color='#BBBCFF').grid(False)
    sns.histplot(adata.obs["n_genes_by_counts"], kde=False, bins=60, ax=axs[2],color='#BBBCFF').grid(False)
    axs[2].set_xlim(0,10000)
    sns.histplot(adata.obs["n_genes_by_counts"][adata.obs["n_genes_by_counts"] < 4000],kde=False,bins=60,ax=axs[3],color='#BBBCFF').grid(False)
    sns.violinplot(adata.obs["pct_counts_mt"],ax=axs[4],color='#BBBCFF').grid(False)
    axs[4].set_ylim(0,15)
    pdf.savefig(fig) # save on the fly
    plt.close() # close figure once saved

pdf.close()



#### Filter genes

From https://www.sc-best-practices.org/preprocessing_visualization/quality_control.html
n_genes_by_counts in .obs is the number of genes with positive counts in a cell,
total_counts is the total number of counts for a cell, this might also be known as library size, and
pct_counts_mt is the proportion of total counts for a cell which are mitochondrial.

 The **MAD** is given by  with  being the respective QC metric of an observation and describes a robust statistic of the variability of the metric. Similar to [Germain et al., 2020], we mark cells as outliers if they differ by 5 MADs which is a relatively permissive filtering strategy. We want to highlight that it might be reasonable to re-assess the filtering after annotation of cells.

In [ ]:
#First, we define a function that takes a metric, i.e. a column in .obs and the number of MADs (nmad) that is still permissive within the filtering strategy.
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * stats.median_abs_deviation(M)) | (
        np.median(M) + nmads * stats.median_abs_deviation(M) < M
    )
    return outlier

For an **AnnData object**: Each row corresponds to a cell with a barcode, and each column corresponds to a gene with a gene id. Furthermore, for each cell and each gene we might have additional metadata, like (1) donor information for each cell, or (2) alternative gene symbols for each gene.

In [ ]:
adata_control_1 #2248 spots with 32285 genes #Nodules 1958
adata_control_2 #3105 spots with 32285 genes #Nodules 2752
adata_Treatment_1 #2490 spots with 32285 genes #Nodules 1849
adata_Treatment_2 #2076 spots with 32285 genes #Nodules 731

In [ ]:
#Instead of choosing the thresholds manually, use the MADs and subset using the indexes (bool)
#sc.pp.filter_cells(adata, min_counts=5000)
#sc.pp.filter_cells(adata, max_counts=50000)
#adata = adata[adata.obs["pct_counts_mt"] < 20].copy()
#print(f"#cells after MT filter: {adata.n_obs}")
#sc.pp.filter_genes(adata, min_cells=20)

#### Which genes to exclude: Xue et al 

In [ ]:
Genesexclude_Xue = pd.read_excel('Xueanalysis_SupTable.xlsx',sheet_name=4, skiprows=2)
Genesexclude_Xue

In [ ]:
# Filter  genes that are detected in less than 10 cells
for adata in [
    adata_control_1,
    adata_control_2,
    adata_Treatment_1,
    adata_Treatment_2
]:
  #filtered out X genes that are detected in less than 10 cells
    sc.pp.filter_genes(adata, min_cells=10,inplace=True)
#filtered out x cells that have less than 200 genes expressed
    sc.pp.filter_cells(adata, min_genes=200,inplace=True)
    


In [ ]:
#Per samples, not in loop remove low quality spots

#--------adata_control_1-----------
#We now apply this function to the log1p_total_counts, log1p_n_genes_by_counts and pct_counts_in_top_20_genes QC covariates each with a threshold of 5 MADs.
print("control_1")
adata_control_1.obs["outlier"] = (
    is_outlier(adata_control_1, "log1p_total_counts", 5)# library size
    | is_outlier(adata_control_1, "log1p_n_genes_by_counts", 5))# number of genes per spot
    #| is_outlier(adata_control_1, "pct_counts_in_top_20_genes", 5) #only spots with 20% of top genes with counts)
print(f"Number of cells after filtering of low quality cells:{adata_control_1.obs.outlier.value_counts()}")

#pct_counts_Mt is filtered with 5 MADs. Additionally, cells with a percentage of mitochondrial counts exceeding 8 % are filtered out.
adata_control_1.obs["mt_outlier"] = is_outlier(adata_control_1, "pct_counts_mt", 5) | (adata_control_1.obs["pct_counts_mt"] > 8)
print(f"Number of cells after filtering of low quality cells:{adata_control_1.obs.mt_outlier.value_counts()}")
    
#Filter according to indexes that passed the MADs thresholds
adata_control_1 = adata_control_1[(~adata_control_1.obs.outlier) & (~adata_control_1.obs.mt_outlier)].copy()
print(f"Number of cells after filtering of low quality cells: {adata_control_1.n_obs}")

#--------adata_control_2 -----------

#We now apply this function to the log1p_total_counts, log1p_n_genes_by_counts and pct_counts_in_top_20_genes QC covariates each with a threshold of 5 MADs.
print("control_2")
adata_control_2.obs["outlier"] = (
    is_outlier(adata_control_2, "log1p_total_counts", 5)# library size
    | is_outlier(adata_control_2, "log1p_n_genes_by_counts", 5))# number of genes per spot
    #| is_outlier(adata_control_1, "pct_counts_in_top_20_genes", 5) #only spots with 20% of top genes with counts)
print(f"Number of cells after filtering of low quality cells:{adata_control_2.obs.outlier.value_counts()}")

#pct_counts_Mt is filtered with 5 MADs. Additionally, cells with a percentage of mitochondrial counts exceeding 8 % are filtered out.
adata_control_2.obs["mt_outlier"] = is_outlier(adata_control_2, "pct_counts_mt", 5) | (adata_control_2.obs["pct_counts_mt"] > 8)
print(f"Number of cells after filtering of low quality cells:{adata_control_2.obs.mt_outlier.value_counts()}")
    
#Filter according to indexes that passed the MADs thresholds
adata_control_2 = adata_control_2[(~adata_control_2.obs.outlier) & (~adata_control_2.obs.mt_outlier)].copy()
print(f"Number of cells after filtering of low quality cells: {adata_control_2.n_obs}")

#--------adata_Treatment_1 -----------

#We now apply this function to the log1p_total_counts, log1p_n_genes_by_counts and pct_counts_in_top_20_genes QC covariates each with a threshold of 5 MADs.
print("Treatment_1")
adata_Treatment_1.obs["outlier"] = (
    is_outlier(adata_Treatment_1, "log1p_total_counts", 5)# library size
    | is_outlier(adata_Treatment_1, "log1p_n_genes_by_counts", 5))# number of genes per spot
    #| is_outlier(adata_control_1, "pct_counts_in_top_20_genes", 5) #only spots with 20% of top genes with counts)
print(f"Number of cells after filtering of low quality cells:{adata_Treatment_1.obs.outlier.value_counts()}")

#pct_counts_Mt is filtered with 5 MADs. Additionally, cells with a percentage of mitochondrial counts exceeding 8 % are filtered out.
adata_Treatment_1.obs["mt_outlier"] = is_outlier(adata_Treatment_1, "pct_counts_mt", 5) | (adata_Treatment_1.obs["pct_counts_mt"] > 8)
print(f"Number of cells after filtering of low quality cells:{adata_Treatment_1.obs.mt_outlier.value_counts()}")
    
#Filter according to indexes that passed the MADs thresholds
adata_Treatment_1 = adata_Treatment_1[(~adata_Treatment_1.obs.outlier) & (~adata_Treatment_1.obs.mt_outlier)].copy()
print(f"Number of cells after filtering of low quality cells: {adata_Treatment_1.n_obs}")

#--------adata_Treatment_2 -----------

#We now apply this function to the log1p_total_counts, log1p_n_genes_by_counts and pct_counts_in_top_20_genes QC covariates each with a threshold of 5 MADs.
print("Treatment_2")
adata_Treatment_2.obs["outlier"] = (
    is_outlier(adata_Treatment_2, "log1p_total_counts", 5)# library size
    | is_outlier(adata_Treatment_2, "log1p_n_genes_by_counts", 5))# number of genes per spot
    #| is_outlier(adata_control_1, "pct_counts_in_top_20_genes", 5) #only spots with 20% of top genes with counts)
print(f"Number of cells after filtering of low quality cells:{adata_Treatment_2.obs.outlier.value_counts()}")

#pct_counts_Mt is filtered with 5 MADs. Additionally, cells with a percentage of mitochondrial counts exceeding 8 % are filtered out.
adata_Treatment_2.obs["mt_outlier"] = is_outlier(adata_Treatment_2, "pct_counts_mt", 5) | (adata_Treatment_2.obs["pct_counts_mt"] > 8)
print(f"Number of cells after filtering of low quality cells:{adata_Treatment_2.obs.mt_outlier.value_counts()}")
    
#Filter according to indexes that passed the MADs thresholds
adata_Treatment_2 = adata_Treatment_2[(~adata_Treatment_2.obs.outlier) & (~adata_Treatment_2.obs.mt_outlier)].copy()
print(f"Number of cells after filtering of low quality cells: {adata_Treatment_2.n_obs}")


In [ ]:
adata_control_1 #2087 spots with 14914 genes
adata_control_2 #2684 spots with 14958 genes
adata_Treatment_1 #2251 spots with 15389 genes
adata_Treatment_2 #2029 spots with 14707 genes

#### Post-QC

In [ ]:
pdf = matplotlib.backends.backend_pdf.PdfPages("output_nodulesonly_postQC.pdf")

for name, adata in [
    ("control_1", adata_control_1),
    ("control_2", adata_control_2),
    ("Treatment_1", adata_Treatment_1),
    ("Treatment_2", adata_Treatment_2),
]:
    fig, axs = plt.subplots(1, 3, figsize=(12, 3))
    fig.suptitle(f"Covariates for filtering: {name}")

    sns.histplot(adata.obs["total_counts"], kde=False, ax=axs[0],color='#BBBCFF').grid(False)
    axs[0].set_xlim(0,85000)
    sns.histplot(adata.obs["n_genes_by_counts"], kde=False, bins=60, ax=axs[1],color='#BBBCFF').grid(False)
    axs[1].set_xlim(0,10000)
    sns.violinplot(adata.obs["pct_counts_mt"],ax=axs[2],color='#BBBCFF')
    axs[2].set_ylim(0,10)
    pdf.savefig(fig) # save on the fly
plt.close() # close figure once saved

pdf.close()


## Normalization

The shifted logarithm works beneficial for stabilizing variance for subsequent dimensionality reduction and identification of differentially expressed genes. Scran was extensively tested and used for batch correction tasks and analytic Pearson residuals are well suited for selecting biologically variable genes and identification of rare cell types.

In [ ]:
#NOT WORKING
import anndata2ri
from rpy2.robjects.packages import importr
from rpy2.robjects import r, pandas2ri
import numpy as np

anndata2ri.activate()
pandas2ri.activate()

def run_sctransform(adata, layer=None, **kwargs):
    if layer:
        mat = adata.layers[layer]
    else:
        mat = adata.X

    # Set names for the input matrix
    cell_names = adata.obs_names
    gene_names = adata.var_names
    r.assign('mat', mat.T)
    r.assign('cell_names', cell_names)
    r.assign('gene_names', gene_names)
    r('colnames(mat) <- cell_names')
    r('rownames(mat) <- gene_names')

    seurat = importr('Seurat')
    r('seurat_obj <- CreateSeuratObject(mat)')

    # Run
    for k, v in kwargs.items():
        r.assign(k, v)
    kwargs_str = ', '.join([f'{k}={k}' for k in kwargs.keys()])
    r(f'seurat_obj <- SCTransform(seurat_obj,vst.flavor="v2", {kwargs_str})')

    # Extract the SCT data and add it as a new layer in the original anndata object
    sct_data = np.asarray(r['as.matrix'](r('seurat_obj@assays$SCT@data')))
    adata.layers['SCT_data'] = sct_data.T
    sct_data = np.asarray(r['as.matrix'](r('seurat_obj@assays$SCT@counts')))
    adata.layers['SCT_counts'] = sct_data.T
    return adata

#### Use Pearson residuals for selection of highly variable genes



In [ ]:
for adata in [
    adata_control_1,
    adata_control_2,
    adata_Treatment_1,
    adata_Treatment_2
]:
    sc.experimental.pp.highly_variable_genes(
        adata, flavor="pearson_residuals", n_top_genes=2000
    )

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
for ax, adata in zip(axes, [adata_control_1,
    adata_control_2,
    adata_Treatment_1,
    adata_Treatment_2]):
    hvgs = adata.var["highly_variable"]

    ax.scatter(
        adata.var["mean_counts"], adata.var["residual_variances"], s=3, edgecolor="none"
    )
    ax.scatter(
        adata.var["mean_counts"][hvgs],
        adata.var["residual_variances"][hvgs],
        c="tab:red",
        label="selected genes",
        s=3,
        edgecolor="none",
    )
    ax.set_xscale("log")
    ax.set_xlabel("mean expression")
    ax.set_yscale("log")
    ax.set_ylabel("residual variance")


    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.yaxis.set_ticks_position("left")
    ax.xaxis.set_ticks_position("bottom")
plt.legend()

In [ ]:
#Apply selection!Skip!
#adata_control_1 = adata_control_1[:, adata_control_1.var["highly_variable"]]
#adata_control_2 = adata_control_2[:, adata_control_2.var["highly_variable"]]

#adata_Treatment_1 = adata_Treatment_1[:, adata_Treatment_1.var["highly_variable"]]
#adata_Treatment_2 = adata_Treatment_2[:, adata_Treatment_2.var["highly_variable"]]

In [ ]:
adata_control_1

Pre-normalization with Pearson Corr: https://github.com/scverse/scanpy-tutorials/blob/main/tutorial_pearson_residuals.ipynb

In [ ]:
# keep raw and depth-normalized counts for later
adata_control_1.layers["raw"] = adata_control_1.X.copy()
adata_control_1.layers["sqrt_norm"] = np.sqrt(
    sc.pp.normalize_total(adata_control_1, inplace=False)["X"]
)

adata_control_2.layers["raw"] = adata_control_2.X.copy()
adata_control_2.layers["sqrt_norm"] = np.sqrt(
    sc.pp.normalize_total(adata_control_2, inplace=False)["X"]
)

adata_Treatment_1.layers["raw"] = adata_Treatment_1.X.copy()
adata_Treatment_1.layers["sqrt_norm"] = np.sqrt(
    sc.pp.normalize_total(adata_Treatment_1, inplace=False)["X"]
)

adata_Treatment_2.layers["raw"] = adata_Treatment_2.X.copy()
adata_Treatment_2.layers["sqrt_norm"] = np.sqrt(
    sc.pp.normalize_total(adata_Treatment_2, inplace=False)["X"]
)


From https://www.sc-best-practices.org/preprocessing_visualization/normalization.html use pearson_residuals as closest to SCT in seurat.
Also from: https://github.com/scverse/scanpy-tutorials/blob/main/tutorial_pearson_residuals.ipynb 

In [ ]:
#from scipy.sparse import csr_matrix, issparse
for adata in [
    adata_control_1,
    adata_control_2,
    adata_Treatment_1,
    adata_Treatment_2
]:
    #sc.pp.normalize_total(adata, inplace=True)
    sc.experimental.pp.normalize_pearson_residuals(adata, inplace=True)
    #sc.pp.log1p(adata,) # Computes X=log(X+1) , where  denotes the natural logarithm unless a different base is given.
    #sc.experimental.pp.highly_variable_genes(adata,flavor="pearson_residuals", n_top_genes=2000, inplace=True)

In [ ]:
adata_Treatment_2.layers["raw"][1,1]

## Manifold embedding and clustering based on transcriptional similarity

Using Pearson coefficients: https://github.com/scverse/scanpy-tutorials/blob/main/tutorial_pearson_residuals.ipynb

In [ ]:
for adata in [
    adata_control_1,
    adata_control_2,
    adata_Treatment_1,
    adata_Treatment_2
]:
    sc.pp.pca(adata, n_comps=50)
    sc.pp.neighbors(adata)
    sc.tl.umap(adata, n_components=50)
    sc.tl.leiden(adata, resolution=0.6, n_iterations=2, directed=False)


In [ ]:
sc.set_figure_params(vector_friendly=False, dpi_save=300) 
end="nodulesonly_UMAP.pdf"
for name, adata in [
    ("control_1", adata_control_1),
    ("control_2", adata_control_2),
    ("Treatment_1", adata_Treatment_1),
    ("Treatment_2", adata_Treatment_2),
]:
    sc.pl.umap(adata, color=[ "leiden"],title=name, wspace=0.4, save='%s_%s' % (name, end))
plt.show()


## Visualization in spatial coordinates

In [ ]:
#plt.rcParams["figure.figsize"] = (8, 8)
#sc.pl.spatial(adata,  color="leiden_res0_5") #save="Treatment_1_leidenres05.pdf")

sc.set_figure_params(vector_friendly=False, dpi_save=300) 
end="nodulesonly_spatial.pdf"
for name, adata in [
    ("control_1", adata_control_1),
    ("control_2", adata_control_2),
    ("Treatment_1", adata_Treatment_1),
    ("Treatment_2", adata_Treatment_2),
]:
    sc.pl.spatial(adata,  color="leiden",title=name, wspace=0.4, size=1.5,save='%s_%s' % (name, end))
    #sc.pl.umap(adata, color=[ "leiden"],title=name, wspace=0.4, save='%s_%s' % (name, end))
plt.show()

## Cluster marker genes

In [ ]:
#from scipy.sparse import csr_matrix, issparse
for adata in [
    adata_control_1,
    adata_control_2,
    adata_Treatment_1,
    adata_Treatment_2
]:
    sc.pp.log1p(adata, layer="raw") # Computes X=log(X+1) , where  denotes the natural logarithm unless a different base is given.
    #sc.experimental.pp.highly_variable_genes(adata,flavor="pearson_residuals", n_top_genes=2000, inplace=True)

In [ ]:
for adata in [
    adata_control_1,
    adata_control_2,
    adata_Treatment_1,
    adata_Treatment_2
]:
    sc.tl.rank_genes_groups(adata, layer="raw",groupby="leiden", method="wilcoxon")

In [ ]:
for adata in [
    adata_control_1,
    adata_control_2,
    adata_Treatment_1,
    adata_Treatment_2
]:
    sc.tl.filter_rank_genes_groups(adata, min_in_group_fraction=0.25,
    max_out_group_fraction=0.25)

In [ ]:
adata_control_1.uns["rank_genes_groups_filtered"]

In [ ]:

#sc.set_figure_params(vector_friendly=False, dpi_save=300) 
#end="DEGpercluster_dotplot.pdf"
for name, adata in [
    ("control_1", adata_control_1),
    ("control_2", adata_control_2),
    ("Treatment_1", adata_Treatment_1),
    ("Treatment_2", adata_Treatment_2),
]:
    sc.pl.rank_genes_groups_dotplot( adata, groupby="leiden",show=True,n_genes=5)#,key="rank_genes_groups_filtered", show=True,n_genes=5)#, save='%s_%s' % (name, end))



In [ ]:
adata_control_1

In [ ]:

sc.pl.rank_genes_groups(adata_control_1)

In [ ]:
sc.pl.rank_genes_groups_dotplot( adata_control_2, groupby="leiden",key='rank_genes_groups_filtered', standard_scale="var",n_genes=5)

In [ ]:
for name, adata in [
    ("control_1", adata_control_1),
    ("control_2", adata_control_2),
    ("Treatment_1", adata_Treatment_1),
    ("Treatment_2", adata_Treatment_2),
]:
    sc.pl.rank_genes_groups_dotplot( adata, groupby="leiden",key='rank_genes_groups_filtered', standard_scale="var",n_genes=5) #, save='%s_%s' % (name, end))

In [ ]:
sc.pl.spatial(adata_control_1, img_key="hires",use_raw=False, vmin=0,color=["Plin2","mt-Co2","Scp2"],save="Examples_clusters_control_1.pdf", size=1.5)

In [ ]:
sc.pl.spatial(adata_control_2, img_key="hires",use_raw=False, color=["Gphn","Cldn7","Serpina1e"],save="Examples_clusters_control_2.pdf", size=1.5)

In [ ]:
sc.pl.spatial(adata_Treatment_2, img_key="hires", color=["Spink1","Ldha","Anxa5"], vmax=[12.5,7,10], vmin=0, size=1.5,save="Examples_clusters_Treatment_2.pdf")

In [ ]:
sc.pl.spatial(adata_Treatment_1, img_key="hires", color=["Isyna1","Fau","Cxcl10"],vmin=0, vmax=[5,5,20],size=1.5,save="Examples_clusters_Treatment_1.pdf")

#### DEG per cluster: dataframes

In [ ]:
#Function to extract DEG per cluster and return a df
def degpercluster(degs,clusters_names):
    degs_list=[] #Initialize list
    for cluster in clusters_names: #Iterate over cluster names
        degpercluster = degs.filter(like=cluster) #filter only columns for cluster
        degs_up = degpercluster[(degpercluster.iloc[:,3] < 0.01) & (degpercluster.iloc[:,4] > 0)] #Filter by pvals_adj <0.05 and logFC> 0
        #degs_dw = degpercluster[degpercluster.iloc[:,3] < 0.05 & degpercluster.iloc[:,4] < 0 ] 
        t=degs_up.iloc[:,0].values
        degs_list.append(t)
    return pd.DataFrame(degs_list).T



In [ ]:
# get deg result
#---------------Control_1
result_control_1 = adata_control_1.uns['rank_genes_groups']
groups_control_1 = result_control_1['names'].dtype.names
degs_control_1 = pd.DataFrame(
    {group + '_' + key: result_control_1[key][group]
    for group in groups_control_1 for key in ['names','scores', 'pvals','pvals_adj','logfoldchanges']})

#---------------Control_2
result_control_2 = adata_control_2.uns['rank_genes_groups']
groups_control_2 = result_control_2['names'].dtype.names
degs_control_2 = pd.DataFrame(
    {group + '_' + key: result_control_2[key][group]
    for group in groups_control_2 for key in ['names','scores', 'pvals','pvals_adj','logfoldchanges']})

#---------------Treatment_1
result_Treatment_1 = adata_Treatment_1.uns['rank_genes_groups']
groups_Treatment_1 = result_Treatment_1['names'].dtype.names
degs_Treatment_1 = pd.DataFrame(
    {group + '_' + key: result_Treatment_1[key][group]
    for group in groups_Treatment_1 for key in ['names','scores', 'pvals','pvals_adj','logfoldchanges']})

#---------------Treatment_2
result_Treatment_2 = adata_Treatment_2.uns['rank_genes_groups']
groups_Treatment_2 = result_Treatment_2['names'].dtype.names
degs_Treatment_2 = pd.DataFrame(
    {group + '_' + key: result_Treatment_2[key][group]
    for group in groups_Treatment_2 for key in ['names','scores', 'pvals','pvals_adj','logfoldchanges']})


In [ ]:
#Extract number of clusters
#---------------Control_1
col_names_control_1 =degs_control_1.columns.values.tolist()
clusters_names_control_1 = list(set([i.split("_")[0] for i in col_names_control_1 ]))
clusters_names_control_1

#---------------Control_2
col_names_control_2 =degs_control_2.columns.values.tolist()
clusters_names_control_2 = list(set([i.split("_")[0] for i in col_names_control_2 ]))
clusters_names_control_2

#---------------Treatment_1
col_names_Treatment_1 =degs_Treatment_1.columns.values.tolist()
clusters_names_Treatment_1 = list(set([i.split("_")[0] for i in col_names_Treatment_1 ]))
clusters_names_Treatment_1

#---------------Treatment_2
col_names_Treatment_2 =degs_Treatment_2.columns.values.tolist()
clusters_names_Treatment_2 = list(set([i.split("_")[0] for i in col_names_Treatment_2 ]))


In [ ]:
#Extract DEG as df per clusters

#---------------Control_1
degs_control_1_DF=degpercluster(degs_control_1,clusters_names_control_1)
degs_control_1_DF.columns = clusters_names_control_1
degs_control_1_DF.head()

#---------------Control_2
degs_control_2_DF=degpercluster(degs_control_2,clusters_names_control_2)
degs_control_2_DF.columns = clusters_names_control_2
#degs_control_2_DF.head()

#---------------Treatment_1
degs_Treatment_1_DF=degpercluster(degs_Treatment_1,clusters_names_Treatment_1)
degs_Treatment_1_DF.columns = clusters_names_Treatment_1
#degs_Treatment_1_DF.head()

#---------------Treatment_2
degs_Treatment_2_DF=degpercluster(degs_Treatment_2,clusters_names_Treatment_2)
degs_Treatment_2_DF.columns = clusters_names_Treatment_2
degs_Treatment_2_DF.head()

In [ ]:
sc.pl.spatial(adata_control_1, img_key="hires", color=["mt-Co2"], size=1.5)

In [ ]:
degs_control_1_DF

In [ ]:
#Write to excel

with pd.ExcelWriter('onlynodules_DEGpercluster.xlsx') as writer:  
    degs_control_1_DF.to_excel(writer, sheet_name='CTL1')
    degs_control_2_DF.to_excel(writer, sheet_name='CTL2')
    degs_Treatment_1_DF.to_excel(writer, sheet_name='Treatment_1')
    degs_Treatment_2_DF.to_excel(writer, sheet_name='Treatment_2')

#degs_control_1_DF.to_excel("DEGpercluster_CTL1.xlsx",sheet_name='CTL1')
#degs_control_2_DF.to_excel("DEGpercluster_CTL2.xlsx",sheet_name='CTL2')


#degs_Treatment_1_DF.to_excel("DEGpercluster_Treatment_1.xlsx",sheet_name='Treatment_1')
#degs_Treatment_2_DF.to_excel("DEGpercluster_Treatment_2.xlsx",sheet_name='Treatment_2')

#### Over-representation analysis (Enrichr API)

In [ ]:
import gseapy as gp
gene_sets="./DEG_MarkersClusters_ZoominSenescentres01_onlyXLhep_PENDING5CLUST.gmt",

In [ ]:
degs_control_1

In [ ]:
# subset up or down regulated genes
degs_sig = degs[degs["1_pvals_adj"] < 0.05]
degs_up = degs_sig[degs["1_logfoldchanges"] > 0]
degs_dw = degs_sig[degs["1_logfoldchanges"] < 0]

In [ ]:
degs_sig.shape

In [ ]:
# Enricr API
enr_up = gp.enrichr(degs_up["0_names"],
                    gene_sets='GO_Biological_Process_2021',
                   organism='mouse', # don't forget to set organism to the one you desired! e.g. Yeast
                 outdir=None) # don't write to disk)
enr2 = gp.enrich(gene_list="./tests/data/gene_list.txt", # or gene_list=glist
                 gene_sets=["./tests/data/genes.gmt", "unknown", kegg ], # kegg is a dict object
                 background=None, # or "hsapiens_gene_ensembl", or int, or text file, or a list of genes
                 outdir=None,
                 verbose=True)

In [ ]:
# trim (go:...)
enr_up.res2d.Term = enr_up.res2d.Term.str.split(" \(GO").str[0]
enr_dw.res2d.Term = enr_dw.res2d.Term.str.split(" \(GO").str[0]

In [ ]:
# concat results
enr_up.res2d['UP_DW'] = "UP"
enr_dw.res2d['UP_DW'] = "DOWN"
enr_res = pd.concat([enr_up.res2d.head(), enr_dw.res2d.head()])

In [ ]:
ax = gp.barplot(enr_res, figsize=(3,5),
                group ='UP_DW',
                title ="GO_BP",
                color = ['b','r'])

In [ ]:
#sc.pl.spatial(adata_Treatment_1, img_key="hires", color=["Cdkn2a","Cdkn1a"], save="Treatment_1_Cdknexp.pdf")

In [ ]:
enr2 = gp.enrich(gene_list="./tests/data/gene_list.txt", # or gene_list=glist
                 gene_sets=["./tests/data/genes.gmt", "unknown", kegg ], # kegg is a dict object
                 background=None, # or "hsapiens_gene_ensembl", or int, or text file, or a list of genes
                 outdir=None,
                 verbose=True)

### Pathway enrichment from Treatment-human-signatures

In [ ]:
#read signature from excel file
#Human Treatment signature
Treatmentsig = pd.read_excel('DEG_TreatmentvsVec_p53mut.xlsx',sheet_name=0) 
Treatmentsigsymbols = Treatmentsig.Symbols[Treatmentsig.logFC >0].tolist()

In [ ]:
#Convert human to mouse symbols https://gseapy.readthedocs.io/en/master/gseapy_example.html
from gseapy import Biomart
bm = Biomart()
# note the dataset and attribute names are different

h2m = bm.query(dataset='hsapiens_gene_ensembl',
               attributes=['ensembl_gene_id','external_gene_name',
                           'mmusculus_homolog_ensembl_gene',
                           'mmusculus_homolog_associated_gene_name'])

In [ ]:
h2m.head()

In [ ]:
#Treatmentsigsymbolsmouse = h2m.mmusculus_homolog_associated_gene_name[h2m['external_gene_name'].isin(Treatmentsigsymbols)].tolist()
Treatmentsigsymbolsmousefiltered = h2m.mmusculus_homolog_associated_gene_name[(h2m['external_gene_name'].isin(Treatmentsigsymbols)) & (~h2m['mmusculus_homolog_associated_gene_name'].isnull())]
len(Treatmentsigsymbolsmousefiltered)

In [ ]:
Treatmentsigdf = pd.DataFrame({"geneset" :'Treatmentsignature',
              "genesymbol" : Treatmentsigsymbolsmousefiltered})
Treatmentsigdf

### Pathway enrichment from scRNAseq senescent -signatures

In [ ]:
#scRNAseq senescent clusters 
#TAKE ONLY TOP 2000 genes
scRNAseq_UP_SenClust = pd.read_excel("Lesscells_wilcox_DEG_MarkersClusters_ZoominSenescentres04_onlyXLhep_NOTstringent_FDR01_minpct01_mindiffpct01_logfc01.xlsx",sheet_name=0, nrows=2000) 
#Treatmentsigsymbols = Treatmentsig.Symbols[Treatmentsig.logFC >0].tolist()
scRNAseq_UP_SenClustdf = scRNAseq_UP_SenClust.melt()
scRNAseq_UP_SenClustdf_filtered =scRNAseq_UP_SenClustdf.dropna()
scRNAseq_UP_SenClustdf_filtered = scRNAseq_UP_SenClustdf_filtered.rename(columns={'variable': 'geneset', 'value': 'genesymbol'}) #Change the name of columns
scRNAseq_UP_SenClustdf_filtered[scRNAseq_UP_SenClustdf_filtered.geneset=="c7"]

### Pathway enrichment from HALLMARKS-HYPOXIA

In [ ]:
from pathlib import Path
def gmt_to_decoupler(pth: Path) -> pd.DataFrame:
    """
    Parse a gmt file to a decoupler pathway dataframe.
    """
    from itertools import chain, repeat

    pathways = {}

    with Path(pth).open("r") as f:
        for line in f:
            name, _, *genes = line.strip().split("\t")
            pathways[name] = genes

    return pd.DataFrame.from_records(
        chain.from_iterable(zip(repeat(k), v) for k, v in pathways.items()),
        columns=["geneset", "genesymbol"],
    )

In [ ]:
HALLMARK_HYPOXIA = gmt_to_decoupler("HALLMARK_HYPOXIA.v2023.2.Mm.gmt")
HALLMARK_HYPOXIA
GOBP_CELLRESPHYPOXIA = gmt_to_decoupler("GOBP_CELLULAR_RESPONSE_TO_HYPOXIA.v2023.2.Mm.gmt")
GOBP_CELLRESPHYPOXIA

#### Concatenate signatures

In [ ]:
#Concatenate Treatmenthumansig with scRNAseq clusters
signatures = pd.concat([Treatmentsigdf, scRNAseq_UP_SenClustdf_filtered,HALLMARK_HYPOXIA,GOBP_CELLRESPHYPOXIA])
signatures[signatures.geneset=="c0"]

In [ ]:
import decoupler #14678+14731+13797+14731
#Run enrichment using two methods: AUCell or overrepresentation
#Results will be saved in obsm under aucell_estimate and  'ora_estimate', 'ora_pvals'
for adata in [
    adata_control_1,
    adata_control_2,
    adata_Treatment_1,
    adata_Treatment_2
]:
    decoupler.run_ora(adata,signatures,source="geneset",min_n=3,n_background= 14484,target="genesymbol",use_raw=False)
    decoupler.run_aucell(adata,signatures,source="geneset",min_n=3,target="genesymbol",use_raw=False)
    #decoupler.run_consensus(adata,Treatmentsigdf,source="geneset",target="genesymbol",use_raw=False)

##### Over-representation

In [ ]:
signatures_colnames = list(adata_Treatment_1.obsm["ora_pvals"])
#Copy results under obs
#For now only overrep results

adata_control_1.obs[signatures_colnames] = adata_control_1.obsm["ora_pvals"][signatures_colnames]
adata_control_2.obs[signatures_colnames] = adata_control_2.obsm["ora_pvals"][signatures_colnames]


adata_Treatment_1.obs[signatures_colnames] = adata_Treatment_1.obsm["ora_pvals"][signatures_colnames]
adata_Treatment_2.obs[signatures_colnames] = adata_Treatment_2.obsm["ora_pvals"][signatures_colnames]


In [ ]:

#adata_Treatment_1.obs['ora_estimate'] = adata_Treatment_1.obsm['ora_estimate']
#adata_Treatment_1.obs['ora_pvals'] = adata_Treatment_1.obsm['ora_pvals']
#adata_Treatment_1.obs['aucell_estimate'] = adata_Treatment_1.obsm['aucell_estimate']

#adata_Treatment_2.obs['ora_estimate'] = adata_Treatment_2.obsm['ora_estimate']
#adata_Treatment_2.obs['ora_pvals'] = adata_Treatment_2.obsm['ora_pvals']
#adata_Treatment_2.obs['aucell_estimate'] = adata_Treatment_2.obsm['aucell_estimate']

In [ ]:
#pvals adjusted and transformation function
def adj_minuslog10 (pval):
    adj_transformed = np.log10(sci.stats.false_discovery_control(pval))*-1
    return(adj_transformed)


In [ ]:
#pvals adjusted

t =adata_control_1.obs[signatures_colnames].apply(lambda x: adj_minuslog10(x), axis=0).add_prefix('adj_hh_log10_')
t_colnames = list(t)
adata_control_1.obs[t_colnames]=t

t =adata_control_2.obs[signatures_colnames].apply(lambda x: adj_minuslog10(x),  axis=0).add_prefix('adj_hh_log10_')
t_colnames = list(t)
adata_control_2.obs[t_colnames]=t

t =adata_Treatment_1.obs[signatures_colnames].apply(lambda x: adj_minuslog10(x), axis=0).add_prefix('adj_hh_log10_')
t_colnames = list(t)
adata_Treatment_1.obs[t_colnames]=t

t =adata_Treatment_2.obs[signatures_colnames].apply(lambda x: adj_minuslog10(x),  axis=0).add_prefix('adj_hh_log10_')
t_colnames = list(t)
adata_Treatment_2.obs[t_colnames]=t


In [ ]:
list(adata_Treatment_1.obs)

In [ ]:
sc.pl.spatial(adata_control_1, color=["adj_hh_log10_HALLMARK_HYPOXIA","adj_hh_log10_c0","adj_hh_log10_c1", "adj_hh_log10_c2","adj_hh_log10_c3","adj_hh_log10_c4","adj_hh_log10_c5","adj_hh_log10_c6","adj_hh_log10_c7"],  vmax=2, vmin=1.3, cmap='mako',size=1.5, save="nodulesonly_Control_1_EnrichmentHYPOXIA_lesscellsscRNAseqSEN8clusters_ORApvaladj.pdf")

In [ ]:
sc.pl.spatial(adata_control_2, color=["adj_hh_log10_HALLMARK_HYPOXIA","adj_hh_log10_c0","adj_hh_log10_c1", "adj_hh_log10_c2","adj_hh_log10_c3","adj_hh_log10_c4","adj_hh_log10_c5","adj_hh_log10_c6","adj_hh_log10_c7"],  vmax=2, vmin=1.3, cmap='mako',size=1.5, save="nodulesonly_Control_2_EnrichmentHYPOXIA_lesscellsscRNAseqSEN8clusters_ORApvaladj.pdf")

In [ ]:
sc.pl.spatial(adata_Treatment_1, color=["adj_hh_log10_HALLMARK_HYPOXIA","adj_hh_log10_c0","adj_hh_log10_c1", "adj_hh_log10_c2","adj_hh_log10_c3","adj_hh_log10_c4","adj_hh_log10_c5","adj_hh_log10_c6","adj_hh_log10_c7"],  vmin=1.3,  cmap='magma',size=1.5, save="nodulesonly_Treatment_1_EnrichmentHYPOXIA_lesscellsscRNAseqSEN8clusters_ORApvaladj.pdf")

In [ ]:
sc.pl.spatial(adata_Treatment_2, color=["adj_hh_log10_HALLMARK_HYPOXIA","adj_hh_log10_c0","adj_hh_log10_c1", "adj_hh_log10_c2","adj_hh_log10_c3","adj_hh_log10_c4","adj_hh_log10_c5","adj_hh_log10_c6","adj_hh_log10_c7"],  vmin=1.3,  cmap='magma',size=1.5, save="nodulesonly_Treatment_2_EnrichmentHYPOXIA_lesscellsscRNAseqSEN8clusters_ORApvaladj.pdf")


In [ ]:
sc.pl.spatial(adata_Treatment_1, color=["adj_hh_log10_HALLMARK_HYPOXIA"], vmin=1.3,vmax=2,  cmap='magma',size=1.5, save="Treatment_1_EnrichmentHYPOXIA_ORApvaladj.pdf")

##### AUCcell

In [ ]:
signatures_colnames = list(adata_Treatment_1.obsm["aucell_estimate"])

#Copy results under obs
#For now only overrep results

adata_control_1.obs[signatures_colnames] = adata_control_1.obsm["aucell_estimate"][signatures_colnames]
adata_control_2.obs[signatures_colnames] = adata_control_2.obsm["aucell_estimate"][signatures_colnames]


adata_Treatment_1.obs[signatures_colnames] = adata_Treatment_1.obsm["aucell_estimate"][signatures_colnames]
adata_Treatment_2.obs[signatures_colnames] = adata_Treatment_2.obsm["aucell_estimate"][signatures_colnames]


In [ ]:
list(adata_Treatment_1.obs)

In [ ]:
sc.pl.spatial(adata_control_1, color=["Treatmentsignature","HALLMARK_HYPOXIA","GOBP_CELLULAR_RESPONSE_TO_HYPOXIA"],  cmap='mako',size=1.5, save="nodulesonly_Control_1_EnrichmentTreatmentsigandHYPOXIA_AUCells.pdf")

In [ ]:
sc.pl.spatial(adata_control_2, color=["Treatmentsignature","HALLMARK_HYPOXIA","GOBP_CELLULAR_RESPONSE_TO_HYPOXIA"],  cmap='mako',size=1.5, save="nodulesonly_Control_2_EnrichmentTreatmentsigandHYPOXIA_AUCells.pdf")

In [ ]:
sc.pl.spatial(adata_Treatment_1, color=["HALLMARK_HYPOXIA","GOBP_CELLULAR_RESPONSE_TO_HYPOXIA","c0", "c2","c3","c4","c5"],  cmap='magma',size=1.5, save="nodulesonly_Treatment_1_EnrichmentHYPOXIA_scRNAseqSENclusters_AUCell.pdf")

In [ ]:
sc.pl.spatial(adata_Treatment_2, color=["HALLMARK_HYPOXIA","GOBP_CELLULAR_RESPONSE_TO_HYPOXIA","c0", "c2","c3","c4","c5"],  cmap='magma',size=1.5, save="nodulesonly_Treatment_2_EnrichmentHYPOXIA_scRNAseqSENclusters_AUCell.pdf")

In [ ]:
#Pending: uses GO or KEGG
#sc.queries.enrich(container=Treatmentsigsymbolsmousefiltered, adata=adata_control_1,groups="0", org="mmusculus")

In [ ]:
#Convert Human gene symbols to Mouse. https://gseapy.readthedocs.io/en/master/gseapy_example.html
# get a dict symbol mappings
#h2m_dict = {}
#for i, row in h2m.loc[:,["external_gene_name", "mmusculus_homolog_associated_gene_name"]].iterrows():
#    if row.isna().any(): continue
#    h2m_dict[row['external_gene_name']] = row["mmusculus_homolog_associated_gene_name"]
# read gmt file into dict
#kegg = gp.read_gmt(path="tests/extdata/enrichr.KEGG_2016.gmt")
#print(kegg['MAPK signaling pathway Homo sapiens hsa04010'][:10])

In [ ]:
#kegg_mouse = {}
#for term, genes in kegg.items():
#    new_genes = []
#    for gene in genes:
#        if gene in h2m_dict:
#            new_genes.append(h2m_dict[gene])
#    kegg_mouse[term] = new_genes
#print(kegg_mouse['MAPK signaling pathway Homo sapiens hsa04010'][:10])

### Pathway enrichment

In [ ]:
#From: https://www.sc-best-practices.org/conditions/gsea_pathway.html?highlight=sc%20queries%20enrich#
# Downloading reactome pathways
from pathlib import Path

#if not Path("c2.cp.reactome.v7.5.1.symbols.gmt").is_file():
#    !wget -O 'c2.cp.reactome.v7.5.1.symbols.gmt' https://figshare.com/ndownloader/files/35233771

In [ ]:
def gmt_to_decoupler(pth: Path) -> pd.DataFrame:
    """
    Parse a gmt file to a decoupler pathway dataframe.
    """
    from itertools import chain, repeat

    pathways = {}

    with Path(pth).open("r") as f:
        for line in f:
            name, _, *genes = line.strip().split("\t")
            pathways[name] = genes

    return pd.DataFrame.from_records(
        chain.from_iterable(zip(repeat(k), v) for k, v in pathways.items()),
        columns=["geneset", "genesymbol"],
    )

In [ ]:
HALLMARK_HYPOXIA = gmt_to_decoupler("HALLMARK_HYPOXIA.v2023.2.Mm.gmt")
HALLMARK_HYPOXIA

In [ ]:
#Run enrichment using two methods: AUCell or overrepresentation
#Results will be saved in obsm under aucell_estimate and  'ora_estimate', 'ora_pvals'
for adata in [
    adata_control_1,
    adata_control_2,
    adata_Treatment_1,
    adata_Treatment_2
]:
    decoupler.run_ora(adata,Treatmentsigdf,source="geneset",n_background= 15000,target="genesymbol",use_raw=False)
    decoupler.run_aucell(adata,Treatmentsigdf,source="geneset",target="genesymbol",use_raw=False)
    #decoupler.run_consensus(adata,Treatmentsigdf,source="geneset",target="genesymbol",use_raw=False)

In [ ]:
sc.pl.spatial(adata_Treatment_2, img_key="hires", color=["n_genes_by_counts","total_counts"], save="Treatment_2_n_genes_by_countsandtotal_counts.pdf")

In [ ]:
sc.pl.spatial(adata_Treatment_2, img_key="hires", color=["n_genes_by_counts","total_counts"], save="Treatment_2_n_genes_by_countsandtotal_counts.pdf")

# Squidpy

In [ ]:
import squidpy as sq

In [ ]:
sq.pl.spatial_scatter(adata_control_1, color="leiden")
img.show(channelwise=True)